In [1]:
import pandas as pd
df1 = pd.read_csv('submission_example.csv')
df2 = pd.read_csv('x_test.csv')
df3 = pd.read_csv('x_train.csv')
df4 = pd.read_csv('y_train.csv')

In [2]:
def norm_y(y):
    new_y = y.rename(columns={'id': 'id', 'Глубина  проникания иглы при 0 °С, [мм-1]': 'depth_0',
                              'Глубина  проникания иглы при 25 °С, [мм-1]': 'depth_25',
                              'Растяжимость  при температуре 0 °С, [см]': 'laxity',
                              'Температура размягчения, [°С]': 'softening', 'Эластичность при 0 °С, [%]': 'elasticity'})
    y=y.drop(['id'], axis=1)
    return new_y

In [3]:
from sklearn.preprocessing import StandardScaler

def norm_x(x):
    x = x.rename(columns={'id': 'id', '% массы <Адгезионная добавка>': 'adhesive_add', '% массы <Базовый битум>': 'bitumen',
       '% массы <Пластификатор>': 'plasticizer', '% массы <Полимер>': 'polymer',
       '% массы <Сшивающая добавка>': 'cross-linking_add', 'Исходная игла при 25С <Базовый битум>': 'bitumen_25',
       'Адгезионная добавка': 'Адгезионная добавка', 'Пластификатор': 'Пластификатор', 'Полимер': 'Полимер',
       'Базовая пенетрация для расчёта пластификатора': 'basic_penetration',
       'Расчёт рецептуры на глубину проникания иглы при 25': 'calculation_25'})
    x=x.drop(['id'], axis=1)
    x_polymer = pd.get_dummies(x['Полимер'])
    x_plasticizer =pd.get_dummies(x['Пластификатор'])
    x_adhesion = pd.get_dummies(x['Адгезионная добавка'])
    x = x.drop(['Полимер', 'Пластификатор', 'Адгезионная добавка'], axis=1)
    x = pd.concat([x, x_polymer, x_plasticizer, x_adhesion], axis=1)
    sc = StandardScaler()
    x['adhesive_add'] = [i[0] for i in sc.fit_transform([[i] for i in x['adhesive_add']])]
    x['bitumen'] = [i[0] for i in sc.fit_transform([[i] for i in x['bitumen']])]
    x['plasticizer'] = [i[0] for i in sc.fit_transform([[i] for i in x['plasticizer']])]
    x['polymer'] = [i[0] for i in sc.fit_transform([[i] for i in x['polymer']])]
    x['cross-linking_add'] = [i[0] for i in sc.fit_transform([[i] for i in x['cross-linking_add']])]
    x['bitumen_25'] = [i[0] for i in sc.fit_transform([[i] for i in x['bitumen_25']])]
    x['basic_penetration'] = [i[0] for i in sc.fit_transform([[i] for i in x['basic_penetration']])]
    x['calculation_25'] = [i[0] for i in sc.fit_transform([[i] for i in x['calculation_25']])]
    return x

In [4]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif, SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.svm import SVR

In [5]:
def lr1(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = SVR(C=10, gamma=0.1).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [6]:
lr1, ml1 = lr1(df2, df3, df4, 'depth_0')

In [7]:

def norm_x(x):
    x = x.rename(columns={'id': 'id', '% массы <Адгезионная добавка>': 'adhesive_add', '% массы <Базовый битум>': 'bitumen',
       '% массы <Пластификатор>': 'plasticizer', '% массы <Полимер>': 'polymer',
       '% массы <Сшивающая добавка>': 'cross-linking_add', 'Исходная игла при 25С <Базовый битум>': 'bitumen_25',
       'Адгезионная добавка': 'Адгезионная добавка', 'Пластификатор': 'Пластификатор', 'Полимер': 'Полимер',
       'Базовая пенетрация для расчёта пластификатора': 'basic_penetration',
       'Расчёт рецептуры на глубину проникания иглы при 25': 'calculation_25'})
    x=x.drop(['id'], axis=1)
    x_polymer = pd.get_dummies(x['Полимер'])
    x_plasticizer =pd.get_dummies(x['Пластификатор'])
    x_adhesion = pd.get_dummies(x['Адгезионная добавка'])
    x = x.drop(['Полимер', 'Пластификатор', 'Адгезионная добавка'], axis=1)
    x = pd.concat([x, x_polymer, x_plasticizer, x_adhesion], axis=1)
#    sc = StandardScaler()
#    x['adhesive_add'] = [i[0] for i in sc.fit_transform([[i] for i in x['adhesive_add']])]
#    x['bitumen'] = [i[0] for i in sc.fit_transform([[i] for i in x['bitumen']])]
#    x['plasticizer'] = [i[0] for i in sc.fit_transform([[i] for i in x['plasticizer']])]
#    x['polymer'] = [i[0] for i in sc.fit_transform([[i] for i in x['polymer']])]
#    x['cross-linking_add'] = [i[0] for i in sc.fit_transform([[i] for i in x['cross-linking_add']])]
#    x['bitumen_25'] = [i[0] for i in sc.fit_transform([[i] for i in x['bitumen_25']])]
#    x['basic_penetration'] = [i[0] for i in sc.fit_transform([[i] for i in x['basic_penetration']])]
#    x['calculation_25'] = [i[0] for i in sc.fit_transform([[i] for i in x['calculation_25']])]
    return x

In [8]:
def lr2(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = LinearRegression().fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [9]:
lr2, ml2 = lr2(df2, df3, df4, 'depth_25')

In [10]:
def lr3(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = GradientBoostingRegressor(random_state=0).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [11]:
lr3, ml3 = lr3(df2, df3, df4, 'laxity')

In [12]:
def lr4(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = GradientBoostingRegressor(random_state=0).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [13]:
lr4, ml4 = lr4(df2, df3, df4, 'softening')

In [14]:
def lr5(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = SVR(C=10, gamma=0.1).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [15]:
lr5, ml5 = lr5(df2, df3, df4, 'elasticity')

In [16]:
id = [0,1,2,3,4,5,6,7,8,9]
res1 = pd.DataFrame()
res1['id'] = id
res1['Глубина  проникания иглы при 0 °С, [мм-1]'] = lr1
res1['Глубина  проникания иглы при 25 °С, [мм-1]'] = lr2
res1['Растяжимость  при температуре 0 °С, [см]'] = lr3
res1['Температура размягчения, [°С]'] = lr4
res1['Эластичность при 0 °С, [%]'] = lr5

In [17]:
res1

id  Глубина  проникания иглы при 0 °С, [мм-1]  \
0   0                                  31.358043   
1   1                                  32.701505   
2   2                                  32.864744   
3   3                                  31.922430   
4   4                                  31.483678   
5   5                                  31.832326   
6   6                                  31.550060   
7   7                                  32.152990   
8   8                                  31.891805   
9   9                                  31.330148   

   Глубина  проникания иглы при 25 °С, [мм-1]  \
0                                   70.310280   
1                                  105.313928   
2                                   65.207972   
3                                   83.283071   
4                                   68.336876   
5                                   60.163738   
6                                   61.126375   
7                                   60.314820   
8                                   96.280451   
9                                   66.075693   

   Растяжимость  при температуре 0 °С, [см]  Температура размягчения, [°С]  \
0                                 15.559729                      61.318046   
1                                 46.005585                      59.020578   
2                                 31.024731                      70.379028   
3                                 43.872450                      69.440191   
4                                 30.860426                      74.110536   
5                                 24.214513                      75.740703   
6                                 19.665540                      75.779686   
7                                 24.275503                      63.982477   
8                                 31.774872                      57.910623   
9                                 22.472624                      63.666113   

   Эластичность при 0 °С, [%]  
0                   72.354338  
1                   75.572623  
2                   73.674706  
3                   75.571972  
4                   74.731652  
5                   75.077443  
6                   70.081767  
7                   76.282189  
8                   75.317536  
9                   76.561643

In [18]:
res1.to_csv(f'v_1_1_jora.csv', index=False)